In [1]:
## imported simple json over json as it is faster
import simplejson as json
import pandas as pd
import timeit
import sys
import re
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize


In [2]:
%matplotlib inline


In [3]:
input_file="gabai.0013.json"

## Reading line by line as the file has erros  and only valid json objects need to be added ot the list, Also the file is large so it saves memmory. 

In [4]:
%%time
ls=[]
with open(input_file) as f:
    for i,line in enumerate(f):
        try:
            c_line=json.loads(line)
            ls.append(c_line)
            if((i+1)%50000==0):
                print(i)
        except:
            print("Error at line number",i)       

49999
99999
149999
199999
249999
299999
349999
399999
449999
499999
549999
599999
649999
699999
749999
799999
849999
Error at line number 872991
Error at line number 872992
Error at line number 872993
Error at line number 872994
Error at line number 872995
Error at line number 872996
Error at line number 872997
Error at line number 872998
Error at line number 872999
Error at line number 873000
Error at line number 873001
Error at line number 873002
Error at line number 873003
Error at line number 873004
Error at line number 873005
Error at line number 873006
Error at line number 873007
Error at line number 873008
Error at line number 873009
Error at line number 873010
Error at line number 873011
Error at line number 873012
Error at line number 873013
Error at line number 873014
Error at line number 873015
Error at line number 873016
Error at line number 873017
Error at line number 873018
Error at line number 873019
Error at line number 873020
Error at line number 873021
Error at line n

# Inspecting Lines where error occured

In [5]:
%%time
with open(input_file) as f:
    lines = f.readlines()
for i in range(872991,873030):
    print(lines[i])

<br />

<b>Warning</b>:  require(/home/forge/gab.ai/bootstrap/../vendor/autoload.php): failed to open stream: No such file or directory in <b>/home/forge/gab.ai/bootstrap/autoload.php</b> on line <b>17</b><br />

<br />

<b>Fatal error</b>:  require(): Failed opening required '/home/forge/gab.ai/bootstrap/../vendor/autoload.php' (include_path='.:/usr/share/php') in <b>/home/forge/gab.ai/bootstrap/autoload.php</b> on line <b>17</b><br />



<br />

<b>Warning</b>:  require(/home/forge/gab.ai/bootstrap/../vendor/autoload.php): failed to open stream: No such file or directory in <b>/home/forge/gab.ai/bootstrap/autoload.php</b> on line <b>17</b><br />

<br />

<b>Fatal error</b>:  require(): Failed opening required '/home/forge/gab.ai/bootstrap/../vendor/autoload.php' (include_path='.:/usr/share/php') in <b>/home/forge/gab.ai/bootstrap/autoload.php</b> on line <b>17</b><br />



<br />

<b>Warning</b>:  require(/home/forge/gab.ai/bootstrap/../vendor/autoload.php): failed to open stream: No

### Now, we have confirmed the fact that we have not missed any useful  message 
### I also tried by changing the range at random to add more sanity checks

In [6]:
%%time
## loading the list into a dataframe now.
df=pd.DataFrame(ls)

CPU times: user 17.6 s, sys: 44.2 s, total: 1min 1s
Wall time: 1min 33s


###  How many messages/gabs are in this dataset?

In [7]:
print("Number of messages is", len(df.index))

Number of messages is 999992


In [12]:
## Explpring the user, we need to create a  new user column with the information of the curtrent dict.
df.loc[1].user

{'id': 189400,
 'is_donor': False,
 'is_investor': False,
 'is_premium': False,
 'is_private': False,
 'is_pro': False,
 'name': 'HenPre13',
 'picture_url': 'https://files.gab.ai/user/59392bee3268b.jpg',
 'username': 'Zwitscher',
 'verified': False}

In [9]:
%%time

##making a new data frame where we have expanded the user column and preffixxed the column name with user_
user_df = pd.concat([df.drop('user', axis=1), pd.DataFrame(df['user'].tolist()).add_prefix('user_')], axis=1)

CPU times: user 10.4 s, sys: 28.2 s, total: 38.6 s
Wall time: 53.8 s


### How many users authored messages are in this dataset?


In [ ]:
## number of users
print("Number of users is", len(user_df['user_id'].unique()))

Number of messages is 15973


###  What are the mean/median/max/min number of messages authored by a user?

In [13]:
### mean/median/max/min number of messages authored by a user
grouped_user_df = user_df.groupby(['user_id']).user_id.count()
print(" User  Message Stats:\n Mean: {} \n Median: {} \n Max: {} \n Min: {}".format( \
      grouped_user_df.mean(), grouped_user_df.median(), \
     grouped_user_df.max() , grouped_user_df.min())) 

 User  Message Stats:
 Mean: 62.60514618418581 
 Median: 4.0 
 Max: 13799 
 Min: 1


### List the 10 users with the most messages, and the number of messages each.


In [14]:
%%time
# top 10 users with messages
top10_df_series = grouped_user_df.nlargest(10)
#search user id in user_df, 
print("Name\t\t\tUser_ID\tCount\n")
for x,y in top10_df_series.iteritems():
    ### can be optimezed look into it
    c_id = (user_df.user_id.values == x).argmax()
    c_user_name = user_df.loc[c_id].user_name
    print('{:<20}\t{}\t{}'.format(c_user_name,x,y))
    


Name			User_ID	Count

Constitutional Drunk	64690	13799
EyeAm               	69650	5996
Ron MacDonald       	100406	5471
#NSFW LeeLee        	102726	4956
Jay                 	169891	4842
Tipol J News        	167852	4816
OpenQuotes 👌🐸       	98146	4678
Dani                	105504	4594
bobofkake-Bro:)     	29492	4404
Lea Morabito        	77796	4230
CPU times: user 27.1 ms, sys: 31.3 ms, total: 58.3 ms
Wall time: 93.6 ms


## URL MINING

In [15]:
"""
the web url matching regex used by markdown
http://daringfireball.net/2010/07/improved_regex_for_matching_urls
https://gist.github.com/gruber/8891611
"""
URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""


In [16]:
%%time

message_list= user_df['body'].tolist()
urls=[]
for message in message_list:
    a=re.findall(URL_REGEX,message.lower())
    urls.extend(a)

### 5 most common urls

In [17]:
url_counter=Counter(urls)
top_5_urls = url_counter.most_common(5)
print("URL  Count")
for url,count in top_5_urls:
    print(url,count)

URL  Count
gab.ai 880
mewe.com/join/lovehaswon 874
https://www.abgeordneten-check.de/kampagnen/stoppt-die-digitale-zensur/startseite/aktion/368383z13871/nc/1/ 441
military.com 384
kek.gg 323


### Text Mining for frequently occuring words

In [18]:
%%time 
all_words=[]
all_alpha_words=[]

stop_words = set(line.strip() for line in open('stop_words.txt'))
stop_words.add("https")
stop_words.add("http")

for message in message_list:
    message=message.lower()
    tokens = word_tokenize(message)
    filtered_words = [w for w in tokens if not w in stop_words]
    alnum_words = [w for w in filtered_words if w.isalnum()]
    alpha_words = [w for w in alnum_words if w.isalpha() ]
    all_words.extend(alnum_words)
    all_alpha_words.extend(alpha_words)
    
words_counter = Counter(all_words)
alpha_words_counter = Couter(all_alpha_words)


### 10 most common alphanumeric words

In [19]:
top_10_words = words_counter.most_common(10)
print(top_10_words)

[('2017', 121158), ('news', 79251), ('trump', 73970), ('watch', 67800), ('youtube', 64861), ('die', 59887), ('07', 57012), ('06', 56197), ('people', 44484), ('der', 42965)]


In [ ]:
### 10 most common words

### Count By Day

In [ ]:
###time zone issue ?
%%time
user_date_time = pd.to_datetime(user_df['created_at'],infer_datetime_format=True)

In [ ]:
%%time
user_date=user_date_time.dt.date

CPU times: user 10.9 s, sys: 930 ms, total: 11.8 s
Wall time: 12.9 s


In [ ]:
date_counts= user_date.value_counts()

In [ ]:
user_date